In [1]:
import numpy as np
import pandas as pd
import glob
import pickle
import cv2
import os
import re

In [2]:
# Function to extract DJI number from a file path
def extract_dji_number(file_path):
    match = re.search(r'DJI_(\d+)', file_path)
    return match.group(1) if match else None

In [3]:
DATE = ['20230309']#, '20230305']
SESSION = ['SE_Lek1']#, 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

registration_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'
tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/Tracking_Fusion'

In [4]:
missing_tracks = []  # To store missing tracks.csv paths

for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            registration_path = f"{registration_directory}/{date}/{session}/{drone}"
            tracking_path = f"{tracking_directory}/{date}/{session}/{drone}/*{session}_{drone}*"

            # Get frames with corresponding anchors
            anchors = sorted(glob.glob(f"{registration_path}/{date}_{session}_{drone}*_Anchored.csv"))

            # Get tracking folders sorted by DJI video name
            tracking_folders = glob.glob(tracking_path)
            tracking_folders = sorted(tracking_folders, key=lambda x: int(re.search(r'DJI_(\d+)', x).group(1)))

            # Get homography matrices from frame to anchor frame
            homography_matrices = sorted(glob.glob(f"{registration_path}/{date}_{session}_{drone}*_homographies.pkl"))

            # Create dictionaries mapping DJI numbers to file paths
            anchors_dict = {extract_dji_number(file): file for file in anchors}
            tracking_dict = {extract_dji_number(file): file for file in tracking_folders}
            homography_dict = {extract_dji_number(file): file for file in homography_matrices}
            
            # Get sorted list of common DJI numbers
            common_dji_numbers = sorted(set(anchors_dict.keys()) & set(tracking_dict.keys()) & set(homography_dict.keys()), key=int)

            for dji in common_dji_numbers:
                print(date, session, drone, dji)

                try:
                    anchor_file = pd.read_csv(anchors_dict[dji])
                    tracks_csv_path = os.path.join(tracking_dict[dji], 'tracks.csv')
                    
                    if not os.path.isfile(tracks_csv_path):
                        raise FileNotFoundError(f"Missing file: {tracks_csv_path}")
                    
                    tracking_file = pd.read_csv(tracks_csv_path)

                    with open(homography_dict[dji], "rb") as f:
                        homography_file = pickle.load(f)

                    tracking_file = tracking_file.merge(anchor_file[['frame', 'best_anchor_frame']], on='frame', how='left')
                    tracking_file['x'] = tracking_file['bb_left'] + tracking_file['bb_width']/2
                    tracking_file['y'] = tracking_file['bb_top'] + tracking_file['bb_height']/2
                    tracking_file['idx'] = tracking_file['master_track_id']
                    tracking_file = tracking_file.drop_duplicates()

                    tracking_file = tracking_file.loc[:,['frame', 'x', 'y', 'idx', 'class_id', 'class_name', 'best_anchor_frame']]

                    for frame in tracking_file['frame'].unique():
                        if frame in homography_file:
                            H = homography_file[frame]
                            matched_points = tracking_file[tracking_file['frame'] == frame][['idx', 'x', 'y']]

                            if not matched_points.empty:
                                src_pts = np.array(matched_points[['x', 'y']], dtype=np.float32).reshape(-1, 1, 2)
                                transformed_pts = cv2.perspectiveTransform(src_pts, H)

                                tracking_file.loc[tracking_file['frame'] == frame, 'transformed_x'] = transformed_pts[:, 0, 0]
                                tracking_file.loc[tracking_file['frame'] == frame, 'transformed_y'] = transformed_pts[:, 0, 1]
                    
                    output_filename = os.path.basename(anchors_dict[dji]).replace('_Anchored.csv', '_Anchored_trajectories.csv')
                    tracking_file.to_csv(os.path.join(os.path.dirname(anchors_dict[dji]), output_filename), index=False, mode='w')

                except FileNotFoundError as e:
                    print(f"Warning: {e}")
                    missing_tracks.append(tracking_dict[dji])
                except Exception as e:
                    print(f"Error processing {date}, {session}, {drone}, {dji}: {e}")
                    continue

20230309 SE_Lek1 P2D4 0791
20230309 SE_Lek1 P2D4 0792
20230309 SE_Lek1 P2D4 0793
20230309 SE_Lek1 P2D4 0794
20230309 SE_Lek1 P2D4 0795
20230309 SE_Lek1 P2D4 0796
20230309 SE_Lek1 P2D4 0797
20230309 SE_Lek1 P2D4 0798
20230309 SE_Lek1 P2D4 0799
20230309 SE_Lek1 P2D4 0800
20230309 SE_Lek1 P2D4 0801
20230309 SE_Lek1 P2D4 0802
20230309 SE_Lek1 P2D4 0803
20230309 SE_Lek1 P2D4 0804
20230309 SE_Lek1 P2D4 0805
20230309 SE_Lek1 P2D4 0806
20230309 SE_Lek1 P2D4 0807
20230309 SE_Lek1 P2D4 0808
20230309 SE_Lek1 P2D4 0809
20230309 SE_Lek1 P2D4 0810
20230309 SE_Lek1 P2D4 0811
20230309 SE_Lek1 P2D4 0812
20230309 SE_Lek1 P2D4 0813
20230309 SE_Lek1 P2D4 0814
